# Parameters

In [ ]:
infile1="cis-meQTL/B-Mem.tsv"
infile2="gwas_catalog/gwas_catalog_cc_no_beta.tsv.gz"
# type1="quant"
# type2="cc"
phe1="dmr_id"
phe2="gwas_id"
has_beta1="no"
has_beta2="no"
N1=110
meta2="gwas_catalog/gwas_catalog_harmonised_metadata.tsv" # should contain columns: ${phe2} and n_case,n_control
outdir="coloc_results"
name="B-Mem.cc.no_beta"

In [ ]:
library(coloc)
suppressMessages(library(tidyverse))
suppressMessages(library(data.table))
library(LDlinkR)

In [ ]:
if (file.exists(outdir)){
    print("outdir already exist")
} else {
    dir.create(file.path(outdir))
    print(paste("create outdir",outdir,sep=' : '))
}

## Read data

In [ ]:
# infile1: quant
QTL1 = fread(infile1,header=F,col.names=c('chrom','position','dmr_id','snp','MAF','pvalues')) %>%
    mutate_at( #mutate_at: perform operation on a selected variable (column)
        "position", #convert position from str to int
        as.numeric
    )

# if (snp1!="snp"){
#     QTL1 <- QTL1 %>% rename(snp  = `snp1`)
# }

# if (pval1!="pvalues"){
#     QTL1 <- QTL1 %>% rename(chrom  = `pval1`)
# }

if (has_beta1=="yes"){ #has beta and varbeta, no need meta1
   QTL1 <- QTL1 %>%
    select(snp, chrom, position, MAF, phe1, beta,varbeta,pvalues) # need N1
} else { # no beta, use pvalue and MAF for quant
   QTL1 <- QTL1 %>%
        select(snp, chrom, position, MAF, phe1, pvalues) # need N1
}

QTL1 <- QTL1 %>% filter(MAF > 0, MAF < 1) %>% drop_na()
QTL1[QTL1$pvalues==0,'pvalues'] <- 5e-324

print(dim(QTL1))
head(QTL1)

In [ ]:
# infile2: cc
QTL2 = fread(infile2) %>%
mutate_at( #mutate_at: perform operation on a selected variable (column)
    "position", #convert position from str to int
    as.numeric
)

if (has_beta2=="no"){ #has beta and varbeta, no need meta1
   QTL2 <- QTL2 %>%
        select(snp, chrom, position, MAF, phe2, pvalues) # need N2 and s (n_case/N2), can be read from meta2

    df_meta2 = fread(meta2) %>%
        select(phe2,n_case,n_control) %>% filter(get(phe2) %in% QTL2[[phe2]]) %>%
        mutate(N=n_case+n_control) %>% mutate(S=n_case / N)

} else { # has beta, use pvalue and MAF for quant
    QTL2 <- QTL2 %>%
        select(snp, chrom, position, MAF,phe2, beta, varbeta,pvalues)
}

QTL2 <- QTL2 %>% filter(MAF > 0, MAF < 1) %>% drop_na()
QTL2[QTL2$pvalues==0,'pvalues'] <- 5e-324

print(dim(QTL2))
head(QTL2)

# lead_SNP

In [ ]:
# Get lead SNP (the SNP with the smallest pvalue for each phenotype)
lead_SNP1 = QTL1 %>%
    group_by(!!!syms(phe1)) %>%
    arrange(pvalues) %>% #arranges the rows in ascending order of the pvalue column, smallest (most significant) p-values are at the top
    distinct(!!!syms(phe1), .keep_all = TRUE)

print(dim(lead_SNP1)) #only selected the SNP with the smallest pvalue for each gene
head(lead_SNP1)

lead_SNP2 = QTL2 %>%
    group_by(!!!syms(phe2)) %>%
    arrange(pvalues) %>% #arranges the rows in ascending order of the pvalue column, smallest (most significant) p-values are at the top
    distinct(!!!syms(phe2), .keep_all = TRUE)

print(dim(lead_SNP2)) #only selected the SNP with the smallest pvalue for each gene
head(lead_SNP2)

# convert QTL dataframe to list

In [ ]:
# Define colocalization pairs
if (has_beta1=="yes"){ #has beta and varbeta, no need meta1
    qtl_list1 = apply( #processes the lead_SNP1 dataframe to create a list of dictionaries
        lead_SNP1 %>%
            mutate(rownames = !!sym(phe1)) %>%
            column_to_rownames("rownames"),
        MARGIN = 1, #applied to each row (MARGIN = 1) of the preprocessed dataframe
        FUN = function(x) {
            result                   = as.list(x)
            result[["MAF"]]          = as.numeric(result[["MAF"]])
            result[["position"]]     = as.numeric(result[["position"]])
            result[["beta"]]    = as.numeric(result[["beta"]])
            result[["varbeta"]] = as.numeric(result[["varbeta"]])
            result[["pvalues"]]  = as.numeric(result[["pvalues"]])
            return(result)
        },
        simplify = FALSE
    ) #convert a dataframe into a list
} else { # no beta, use pvalue and MAF for quant
    qtl_list1 = apply( #processes the lead_SNP1 dataframe to create a list of dictionaries
        lead_SNP1 %>%
            mutate(rownames = !!sym(phe1)) %>%
            column_to_rownames("rownames"),
        MARGIN = 1, #applied to each row (MARGIN = 1) of the preprocessed dataframe
        FUN = function(x) {
            result                   = as.list(x)
            result[["MAF"]]          = as.numeric(result[["MAF"]])
            result[["position"]]     = as.numeric(result[["position"]])
            result[["pvalues"]]  = as.numeric(result[["pvalues"]])
            return(result)
        },
        simplify = FALSE
    ) #convert a dataframe into a list
}

print(length(qtl_list1))
head(qtl_list1)


In [ ]:
# for test: to run faster, only select top 100 qtl_list1
# qtl_list1 <- head(qtl_list1,50)
# print(length(qtl_list1))
# head(qtl_list1)

# overlapped_SNP2 (qtl_list2) whose SNPs overlapped with qtl_list1

In [ ]:
# get the overlapped SNP from infile2
qtl_list2 = lapply( #lapply function applies a function (specified by FUN) to each element of a list.
    X = qtl_list1,
    FUN = function(x) {
        QTL2 %>% filter(snp == x[["snp"]]) #chrom == x[["chrom"]], position - x[["position"]] <= 1000000, x[["position"]]-position <= 1000000
    }
) # a list, each element is a dataframe whose snp equal to x[['snp']] for each x in qtl_list1
qtl_list2 <- qtl_list2[sapply(qtl_list2, function(x) length(x[["snp"]]) > 0)]
print(length(qtl_list2))
head(qtl_list2)

# only keep overlapped QTL1
qtl_list1 <- qtl_list1[names(qtl_list2)]

In [ ]:
if(length(qtl_list2) ==0 ) {
    print("No overlap")
    quit()
}

# LD
For each phe1, find only 1 snp2 in qtl_list2 df having largest LD with snp1

In [ ]:
# lead_phe2 = mapply( #mapply function applies a function (specified by FUN) to corresponding elements from multiple lists
#         FUN = function(qtl1, qtl2) { #defines the function that will be applied to each pair of elements (qtl1 and meqtl_overlap) from the corresponding lists
#             if(length(qtl2[["snp"]]) == 0) {
#                 return(0)
#             }
#             if(length(unique(qtl2[[phe2]])) == 1) {
#                 return(1)
#             } else { #multiple phe2
#                 lead_snp1 = qtl1[["snp"]]
#                 query2 <- lead_SNP2 %>% filter(get(phe2) %in% qtl2[[phe2]],chrom==qtl1$chrom) %>% arrange(pvalues,MAF)
#                 query2 <- query2[duplicated(query2$snp),]
#                 lead_snp2_query <- query2$snp
#                 if(length(lead_snp2_query) == 0){
#                     # ld_with_lead_snp1 <- c(0)
#                     return(0)
#                 } else if (length(lead_snp2_query) == 1) {
#                     return(1)
#                 } else if (lead_snp1 %in% lead_snp2_query) {
#                     return("yes")
#                 } else {
#                     return("N")
#                 }
#             }
#         },
#         qtl_list1,
#         qtl_list2,
#         SIMPLIFY = FALSE
#     )

In [ ]:
# Calculate LD
if (has_beta2=="yes"){
    lead_phe2 = mapply( #mapply function applies a function (specified by FUN) to corresponding elements from multiple lists
        FUN = function(qtl1, qtl2) { #defines the function that will be applied to each pair of elements (qtl1 and meqtl_overlap) from the corresponding lists
            if(length(qtl2[["snp"]]) == 0) {
                return(list(
                    phe2        = NA,
                    beta2   = NA,
                    varbeta2      = NA,
                    pvalues2 = NA,
                    snp2 = NA
                ))
            }
            if(length(unique(qtl2[[phe2]])) == 1) {
                return(
                    list(
                        phe2         = qtl2[[phe2]][1],
                        beta2    = qtl2[["beta"]][1],
                        varbeta2 = qtl2[["varbeta"]][1],
                        pvalues2  = qtl2[["pvalues"]][1],
                        snp2 = qtl2[["snp"]][1]
                    )
                )
            } else { #multiple phe2
                lead_snp1 = qtl1[["snp"]]
                query2 <- lead_SNP2 %>% filter(get(phe2) %in% qtl2[[phe2]],chrom==qtl1$chrom) %>% arrange(pvalues,MAF)
                query2 <- query2[duplicated(query2$snp),]
                lead_snp2_query <- query2$snp
                # simplified apply (sapply) is a function used to apply a function to each element, row, or column of a vector, matrix, data frame, or array and return the results in a simplified way
                if(length(lead_snp2_query) == 0){
                    # ld_with_lead_snp1 <- c(0)
                    return(list(
                            phe2  = NA,
                            beta2   = NA,
                            varbeta2      = NA,
                            pvalue2 = NA,
                            snp2 = NA
                        ))
                } else if (length(lead_snp2_query) == 1) {
                    return(list(
                            phe2  = query2[[phe2]][1],
                            beta2   = NA,
                            varbeta2      = NA,
                            pvalue2 = query2$pvalues[1],
                            snp2 = query2$snp[1]
                        ))
                } else if (lead_snp1 %in% lead_snp2_query) {
                    return(list(
                            phe2  = query2[query2$snp==lead_snp1,][[phe2]][1],
                            beta2   = query2[query2$snp==lead_snp1,]$beta[1],
                            varbeta2 = query2[query2$snp==lead_snp1,]$varbeta[1],
                            pvalue2 = query2[query2$snp==lead_snp1,]$pvalues[1],
                            snp2 = lead_snp1
                        ))
                } else {
                    ld_with_lead_snp1 = sapply( #Uses sapply and LDlinkR package to calculate LD (R2) between each selected meSNP and the lead eSNP.
                        X = lead_snp2_query,
                        FUN = function(x) {
                            if (length(x) == 0) {
                                return(0)
                            } else {
                                tryCatch({
                                    ld_matrix = LDlinkR::LDpair(
                                        var1  = x,
                                        var2  = lead_snp1,
                                        pop   = "CEU",
                                        #  https://ldlink.nih.gov/?tab=apiaccess
                                        token = "YOUR_TOKEN"
                                    )
                                    return(ld_matrix[["r2"]][1]) #calculate the LD between two SNP
                                },
                                error = function(error) {
                                    print(paste(x, lead_snp1, error, sep = "\t"))
                                    return(0)
                                })
                            }
                        },
                        simplify = "array"
                    )
                }
                
                tryCatch({
                    max_ld = max(ld_with_lead_snp1,na.rm=T)
                    if(max_ld == 0 | is.na(max_ld)) {
                        return(list(
                            phe2  = NA,
                            beta2   = NA,
                            varbeta2  = NA,
                            pvalues2 = NA,
                            snp2 = NA
                        ))
                    } else { # If LD is valid, it identifies the SNP (max_ld_snp) corresponding to the maximum LD and extracts the lead phe2 (max_ld_phe2), beta (max_ld_beta), varbeta (max_ld_varbeta), and pvalue (max_ld_pvalue) associated with that SNP.
                        max_ld_snp = names(ld_with_lead_snp1)[which(ld_with_lead_snp1 == max_ld)]
                        max_ld_phe2   = (lead_SNP2 %>% filter(snp %in% max_ld_snp) %>% arrange(pvalues))[[phe2]][1]
                        max_ld_beta    = (lead_SNP2 %>% filter(snp %in% max_ld_snp) %>% arrange(pvalues))$beta[1]
                        max_ld_varbeta = (lead_SNP2 %>% filter(snp %in% max_ld_snp) %>% arrange(pvalues))$varbeta[1]
                        max_ld_pvalue  = (lead_SNP2 %>% filter(snp %in% max_ld_snp) %>% arrange(pvalues))$pvalues[1]
                        return(
                            list(
                                phe2        = max_ld_phe2,
                                beta2   = max_ld_beta,
                                varbeta2      = max_ld_varbeta,
                                pvalues2 = max_ld_pvalue,
                                snp2 = max_ld_snp
                            )
                        )
                    }
                },
                error = function(error) {
                    print(error)
                    return(list(
                            phe2  = NA,
                            beta2   = NA,
                            varbeta2  = NA,
                            pvalues2 = NA,
                            snp2 = NA
                        ))
                })
            }
        },
        qtl_list1,
        qtl_list2,
        SIMPLIFY = FALSE
    ) #Finally, it returns a list containing the lead max_ld_phe2, beta, varbeta, and pvalue for each
} else { #no beta: snp, chrom, position, MAF, phe2, pvalues
    lead_phe2 = mapply( #mapply function applies a function (specified by FUN) to corresponding elements from multiple lists
        FUN = function(qtl1, qtl2) { #defines the function that will be applied to each pair of elements (qtl1 and qtl2) from the corresponding lists
            # no overlap: snp, chrom, position, MAF, phe2, pvalues
            if(length(qtl2[["snp"]]) == 0) {
                return(list(
                    phe2        = NA,
                    pvalue2 = NA,
                    snp2 = NA
                ))
            }
            if(length(unique(qtl2[[phe2]])) == 1) {
                return(
                    list(
                        phe2         = qtl2[[phe2]][1],
                        pvalue2  = qtl2[["pvalues"]][1],
                        snp2 = qtl2[["snp"]][1]
                    )
                )
            } else { #multiple phe2
                lead_snp1 = qtl1[["snp"]]
                query2 <- lead_SNP2 %>% filter(get(phe2) %in% qtl2[[phe2]],chrom==qtl1$chrom) %>% arrange(pvalues,MAF)
                query2 <- query2[duplicated(query2$snp),]
                lead_snp2_query <- query2$snp
                # simplified apply (sapply) is a function used to apply a function to each element, row, or column of a vector, matrix, data frame, or array and return the results in a simplified way
                if(length(lead_snp2_query) == 0){
                    # ld_with_lead_snp1 <- c(0)
                    return(list(
                            phe2  = NA,
                            pvalue2 = NA,
                            snp2 = NA
                        ))
                } else if (length(lead_snp2_query) == 1) {
                    return(list(
                            phe2  = query2[[phe2]][1],
                            pvalue2 = query2$pvalues[1],
                            snp2 = query2$snp[1]
                        ))
                } else if (lead_snp1 %in% lead_snp2_query) {
                    return(list(
                            phe2  = query2[query2$snp==lead_snp1,][[phe2]][1],
                            pvalue2 = query2[query2$snp==lead_snp1,]$pvalues[1],
                            snp2 = lead_snp1
                        ))
                } else {
                    ld_with_lead_snp1 = sapply( #Uses sapply and LDlinkR package to calculate LD (R2) between each selected meSNP and the lead eSNP.
                        X = lead_snp2_query,
                        FUN = function(x) {
                            if (length(x) == 0) {
                                return(0)
                            } else {
                                tryCatch({
                                    ld_matrix = LDlinkR::LDpair(
                                        var1  = x,
                                        var2  = lead_snp1,
                                        pop   = "CEU",
                                        # https://ldlink.nih.gov/?tab=apiaccess
                                        token = "YOUR_TOKEN"
                                    )
                                    return(ld_matrix[["r2"]][1]) #calculate the LD between two SNP
                                },
                                error = function(error) {
                                    print(paste(x, lead_snp1, error, sep = "\t"))
                                    return(0)
                                })
                            }
                        },
                        simplify = "array"
                    )
                }
                
                tryCatch({
                    max_ld = max(ld_with_lead_snp1,na.rm=T)
                    if(max_ld == 0 | is.na(max_ld)) {
                        return(list(
                            phe2  = NA,
                            pvalue2 = NA,
                            snp2 = NA
                        ))
                    } else { # If LD is valid, it identifies the SNP (max_ld_snp) corresponding to the maximum LD and extracts the lead phe2 (max_ld_phe), beta (max_ld_beta), varbeta (max_ld_varbeta), and pvalue (max_ld_pvalue) associated with that SNP.
                        max_ld_snp = names(ld_with_lead_snp1)[which(ld_with_lead_snp1 == max_ld)]
                        max_ld_phe   = (lead_SNP2 %>% filter(snp %in% max_ld_snp) %>% arrange(pvalues))[[phe2]][1]
                        max_ld_pvalue  = (lead_SNP2 %>% filter(snp %in% max_ld_snp) %>% arrange(pvalues))$pvalues[1]
                        return(
                            list(
                                phe2        = max_ld_phe,
                                pvalue2 = max_ld_pvalue,
                                snp2 = max_ld_snp
                            )
                        )
                    }
                },
                error = function(error) {
                    print(error)
                    return(
                            list(
                                phe2        = NA,
                                pvalue2 = NA,
                                snp2 = NA
                            )
                        )
                })
            }
        },
        qtl_list1,
        qtl_list2,
        SIMPLIFY = FALSE
    )
}

head(lead_phe2)


In [ ]:
lead_phe2 <- lead_phe2[sapply(lead_phe2, function(x) ! is.na(x["phe2"]))]
qtl_list1 <- qtl_list1[names(lead_phe2)]
qtl_list2 <- qtl_list2[names(lead_phe2)]

In [ ]:
head(lead_phe2)

# Run coloc

In [ ]:
SNP1 = sapply(
    X = names(lead_phe2), #names of lead_phe2 is the same as qtl_list1 (phe1)
    FUN = function(p1) {
        QTL1 %>% filter(get(phe1) == p1, MAF > 0, MAF < 1)
    },
    simplify = FALSE,
    USE.NAMES = TRUE
)
# head(SNP1)
length(SNP1)

In [ ]:
# SNP2 = lapply(
#     X = SNP1,
#     FUN = function(snp1) {
#         QTL2 %>% filter(snp %in% snp1[["snp"]]) # get the meQTl using snp as anchor to connect with genes from eQTL
#     }
# )
# head(SNP2)
SNP2 = lapply(
    X = names(lead_phe2),
    FUN = function(p) {
        QTL2 %>% filter(snp %in% SNP1[[p]][["snp"]],get(phe2)==lead_phe2[[p]][['phe2']]) # get the meQTl using snp as anchor to connect with genes from eQTL
    }
)
names(SNP2) <- names(lead_phe2)
length(SNP2)

In [ ]:
# Run coloc
coloc_result = mapply(
    FUN = function(df_1, df_2, n_1, type_1 = "quant", type_2 = "cc") {
    #df1 is eqtl dataframe for each gene, df2 is meQTL for each gene (using snp as anchor)
        if (nrow(df_1) == 0 | nrow(df_2) == 0) {
            return(list(n_snps = 0,PP3    = 0,PP4    = 0))
            }
        df_1 <- df_1 %>% # rename(MAF = maf, p = pvalue) %>%
            arrange(pvalues) %>% distinct(snp, .keep_all = TRUE) # coloc要求不能有重复的SNP，所以只保留更显著的

        if (has_beta1=="yes"){
            df_1 <- df_1 %>%
                select(snp, position, pvalues, beta, varbeta, MAF) #has beta and varbeta, no need meta1
        } else {
            df_1 <- df_1 %>%
                select(snp, position, pvalues, MAF) # need N1,# no beta, use pvalue and MAF for quant
        }
        df_1 <- df_1 %>% filter(!is.na(MAF)) %>% as.list()
        df_1[["N"]] = n_1
        df_1[["type"]] = type_1

        df_2 <- df_2 %>% #rename(MAF = maf, p = pvalue) %>%
            arrange(pvalues) %>% distinct(snp, .keep_all = TRUE)
        if (has_beta2=="no"){
            sample_size = structure(df_meta2$N,names=df_meta2[[phe2]])
            case_fraction = structure(df_meta2$S,names=df_meta2[[phe2]])
            df_2 <- df_2 %>%
                select(snp, position, pvalues, MAF, phe2) # #has beta and varbeta, no need meta1, need N2 and s (n_case/N2), can be read from meta2
            df_2 <- df_2 %>% filter(!is.na(MAF)) %>% as.list()
            df_2[["N"]] = as.numeric(sample_size[df_2[[phe2]]][1]) #length(unique(phe2)) should be equal to 1
            df_2[["s"]] = as.numeric(case_fraction[df_2[[phe2]]][1])
        } else { # has beta, use pvalue and MAF for quant
            df_2 <- df_2 %>%
                select(snp, position, pvalues, beta, varbeta, MAF) %>%
                filter(!is.na(MAF)) %>% as.list()
        }

        df_2[["type"]] = type_2
        df_2 <- df_2[names(df_2) != phe2]

        if (length(df_1[["snp"]])== 0 | length(df_2[["snp"]]) == 0) {
            return(list(n_snps = 0,PP3    = 0,PP4    = 0))
            }
        tryCatch({
            if (is.null(coloc::check_dataset(df_1)) & is.null(coloc::check_dataset(df_2))) {
                invisible(capture.output({
                    coloc_result = coloc::coloc.abf(
                        dataset1 = df_1,
                        dataset2 = df_2)
                        }))
                return(
                    list(
                        n_snps = coloc_result[["summary"]][["nsnps"]],
                        PP3    = coloc_result[["summary"]][["PP.H3.abf"]],
                        PP4    = coloc_result[["summary"]][["PP.H4.abf"]]
                    )
                )
            } else {
                return(
                    list(
                        n_snps = 0,
                        PP3    = 0,
                        PP4    = 0
                    )
                )
            }},
            error = function(error) {
                print(error)
                return(
                    list(
                        n_snps = 0,
                        PP3    = 0,
                        PP4    = 0
                    )
                )
            })
    },
    df_1      = SNP1,
    df_2      = SNP2, #qtl_list2,
    n_1       = N1, # sample size
    SIMPLIFY  = FALSE,
    USE.NAMES = TRUE
)

In [ ]:
# all(names(qtl_list1)==names(coloc_result))

In [ ]:
final_coloc_result_list = mapply(
    FUN = function(qtl1, qtl2_lead_phe, coloc_result) {
        return(c(
            qtl1,
            qtl2_lead_phe,
            coloc_result
        ))
    },
    qtl1  = qtl_list1, # eQTL (snp,chr,position,ref,alt,maf,gene,beta,varbeta,pvalue) for each gene
    qtl2_lead_phe = lead_phe2, #meQTL (lead probe: probe,beta_meqtl,varbeta_meqtl,pvalue_meqtl), for each gene in eQTL, get the esnp (corresponding to this gene), then get the probe (from meQTL) using snp as anchor, then get the lead_me_snp with the largest LD using probe as anchor.
    coloc_result      = coloc_result,
    SIMPLIFY          = FALSE
)
head(final_coloc_result_list)

In [ ]:
final_coloc_result_table = do.call(rbind, final_coloc_result_list) %>%
    as.data.frame() %>%
    mutate_at(
        .vars = vars(c("snp", "chrom", phe1, phe2,"snp2")),
        .funs = as.character
    ) %>%
    mutate_at(
        .vars = vars(-c("snp", "chrom", phe1, phe2,"snp2")),
        .funs = as.numeric
    ) %>%
    mutate_all(
        .funs = function(x) {
            ifelse(is.na(x) | x == "NA", NA, x)
        }
    ) %>%
    mutate(PP4 = ifelse(is.na(phe2), 0, PP4)) %>%
    arrange(desc(PP4))
head(final_coloc_result_table)

In [ ]:
sorted_df <- final_coloc_result_table %>% arrange(desc(PP4))
head(sorted_df)
dim(sorted_df)

In [ ]:
write.table(sorted_df,file=paste(outdir,paste0(name,".txt"),sep="/"),
            quote=FALSE,col.names=T,row.names=T,sep='\t')

In [ ]:
save(SNP1, SNP2, sorted_df, file = paste(outdir,paste0(name,".rda"),sep="/"))